spark components at high level:
> low level api : RDD, distributed variable

> structured api : dataframes, datasets, SQL

> Libraries and ecosystem , streaming and advanced analytics

>

used for parallel data processing only, data is not stored anywehre.
1. spark SQL
2. streaming
3. MLlib
4. Graphx
5. SparkR
   spark is written in scala

## Main components
   ### spark session (master node/Driver) >> cluster manager >> executor/worker node

  this processing happens on unified (ANYONE CAN USE SAME COMPUTER)computer cluster
  master node > slave node
              > slave node

data is processed in RAM, using lazy evaluation


previously data format was tabular
then file, text, csv, image, video, audio format data was introduced
structured data: key value system

## 3 v's of big data > 
1. velocity > 1 sec, 1 hour
2. volume > 5GB, 10TB
3. variety > structured, semi structured, unstructured

ETL, ELT

issues of handling big data > storage, processing (ram, cpu)

## 2 options : 
1. Monolithic approach > vertical scaling, expensive, low availability
2. Distributed approach > horizonatl scaling, economical, highly available

## HADOOP :
1. hadoop also uses ram, it is slow because i writes data to disk and reads back from disk to memory.
if it does not write and read back from disk then it wont matter

2. Batch data processing
3. difficult to write code in hadoop
4. more secure with kerberos authentication, ACL (access control list) authorization
YARN > kerberos
5. fault tolerance >. it has block of data and replication factor to handle the failure




## SPARK :
1. spark is faster because it does all the computation in ram (in memory), it writes data in memory in ram of the executor
2. both streaming and batch data processing
3. easy to write code, also can be written inn depth
4. uses HDFS storage, ,ACL > kerberos > less secure
5. uses directed acyclic graph DAG > RDD which is immutable, it knows how the process is created using dag

## Libraries for high level api:
datatframe spark sql, streaming, ML lib, GraphX >> dataframe
## low level api: spark core
python, java, R, scala >> RDD (not used nowadays)

everything is under spark engine


spark core >> java wrapper >> python wrapper|
there is no pyspark driver


if python udf used in pyspark code, > udf is not optimized, we need python worker to execute them, which is time consuming
> so use inbuilt functions as much as possible


## TRANSFORMATION in pyspark 

> narrow dependency transformation

1. transformations which are not dependent on any other partition called narrow dependency
2. example: filter, select, union, map() etc
3. example: show me people with age > 18
   
> wide dependency transformation
1. this needs data to be used from 1 partition to other and it is dependent on other partitions as well
2. avoid wide transformations as much as possible because they are expensive to execute
3. example: sum of everyones total salary, where data is partitioned, here data needs to be shuffled

   



> jobs created  for each  action in spark lineage
> 
> every action will have one dag  and every job will have its own dag 

In [2]:
from pyspark.sql import SparkSession



In [3]:
spark = SparkSession.builder.appName("flights").getOrCreate() 
spark

In [4]:
flights = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("../data/flights.csv")


In [5]:
flights.columns

['Year',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'DepTime',
 'CRSDepTime',
 'ArrTime',
 'CRSArrTime',
 'UniqueCarrier',
 'FlightNum',
 'TailNum',
 'ActualElapsedTime',
 'CRSElapsedTime',
 'AirTime',
 'ArrDelay',
 'DepDelay',
 'Origin',
 'Dest',
 'Distance',
 'TaxiIn',
 'TaxiOut',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay']

In [19]:

flights.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335

In [6]:
flight_data_partition = flights.repartition(3)



In [21]:
las_data = flights.filter("Dest=='LAS'")


In [28]:
ind_las_data = las_data.filter(las_data.Origin == 'IND')


In [35]:
total_flight_ind_las = ind_las_data.groupby(ind_las_data.Dest).sum()


 ### spark does lazy evaluation for above all transformations

In [36]:
total_flight_ind_las.show()

+----+---------+----------+---------------+--------------+---------------+---------------+--------------+-------------------+-------------+--------------+-------------+
|Dest|sum(Year)|sum(Month)|sum(DayofMonth)|sum(DayOfWeek)|sum(CRSDepTime)|sum(CRSArrTime)|sum(FlightNum)|sum(CRSElapsedTime)|sum(Distance)|sum(Cancelled)|sum(Diverted)|
+----+---------+----------+---------------+--------------+---------------+---------------+--------------+-------------------+-------------+--------------+-------------+
| LAS|   116464|        58|            986|           232|          84940|          91320|         70651|              14500|        92278|             0|            0|
+----+---------+----------+---------------+--------------+---------------+---------------+--------------+-------------------+-------------+--------------+-------------+



>
> spark ui has all the details about above transformations and actions 